In [1]:
# =============================
# 1. IMPORTS
# =============================
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler


In [2]:
# =============================
# 2. PATHS & RAW LOAD
# =============================
TRAIN_PATH = "/kaggle/input/financial-risk-profiling/train_updated.csv"
TEST_PATH  = "/kaggle/input/financial-risk-profiling/test_updated.csv"

train_raw = pd.read_csv(TRAIN_PATH)
test_raw  = pd.read_csv(TEST_PATH)

print("raw train shape:", train_raw.shape)
print("raw test shape :", test_raw.shape)

# Quick debug of the target in raw
print("\n[DEBUG] raw RiskFlag distribution:")
print(train_raw["RiskFlag"].value_counts())


raw train shape: (204277, 18)
raw test shape : (51070, 17)

[DEBUG] raw RiskFlag distribution:
RiskFlag
0    180524
1     23753
Name: count, dtype: int64


In [3]:
# =============================
# 3. SEPARATE TARGET & PROFILEID (preserve index)
# =============================
TARGET = "RiskFlag"
IDCOL = "ProfileID"

# Make copies and reset index to ensure safe operations but keep original index in a column
train = train_raw.copy().reset_index(drop=False).rename(columns={"index": "orig_index"})
test  = test_raw.copy().reset_index(drop=False).rename(columns={"index": "orig_index"})

# Extract y directly from raw train (so we never lose true labels)
y = train_raw[TARGET].reset_index(drop=True)

# Save ProfileID separately for submission and also keep orig_index for sanity checks
train_profile = train[IDCOL].reset_index(drop=True)
test_profile  = test[IDCOL].reset_index(drop=True)

# Build X by dropping the target (but keep ProfileID out of modelling features)
X = train.drop(columns=[TARGET, IDCOL])  # includes orig_index column retained
test_X = test.drop(columns=[IDCOL])

print("X shape:", X.shape, "test_X shape:", test_X.shape, "y shape:", y.shape)
# DEBUG
print("\n[DEBUG] First rows of X (to inspect):")
display(X.head())


X shape: (204277, 17) test_X shape: (51070, 17) y shape: (204277,)

[DEBUG] First rows of X (to inspect):


,orig_index,ApplicantYears,AnnualEarnings,RequestedSum,TrustMetric,WorkDuration,ActiveAccounts,OfferRate,RepayPeriod,DebtFactor,QualificationLevel,WorkCategory,RelationshipStatus,OwnsProperty,FamilyObligation,FundUseCase,JointApplicant
0,0,18,137576,209136,846,26,2,10.47,60,0.81,High School,Self-employed,Single,Yes,No,Business,No
1,1,47,57194,5970,748,30,2,19.72,36,0.73,High School,Unemployed,Divorced,No,Yes,Education,No
2,2,26,84328,95065,453,7,2,24.25,12,0.45,Master's,Self-employed,Married,No,No,Other,Yes
3,3,53,49795,229582,533,107,3,14.44,60,0.17,Bachelor's,Self-employed,Single,Yes,No,Auto,Yes
4,4,49,115450,22072,840,0,4,24.48,12,0.11,Bachelor's,Part-time,Single,No,Yes,Education,Yes


In [4]:
# =============================
# 4. NUMERIC / CATEGORICAL SPLIT
# =============================
# Exclude orig_index column from numeric features
exclude_cols = ["orig_index"]
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in exclude_cols]

categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

print("Numeric cols:", numeric_cols)
print("Categorical cols:", categorical_cols)


Numeric cols: ['ApplicantYears', 'AnnualEarnings', 'RequestedSum', 'TrustMetric', 'WorkDuration', 'ActiveAccounts', 'OfferRate', 'RepayPeriod', 'DebtFactor']
Categorical cols: ['QualificationLevel', 'WorkCategory', 'RelationshipStatus', 'OwnsProperty', 'FamilyObligation', 'FundUseCase', 'JointApplicant']


In [5]:
# =============================
# 5. MISSING VALUE HANDLING
# =============================
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

# Fit on train features and transform both train and test feature sets
X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])
test_X[numeric_cols] = num_imputer.transform(test_X[numeric_cols])

X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
test_X[categorical_cols] = cat_imputer.transform(test_X[categorical_cols])

print("[DEBUG] Missing-value handling complete.")


[DEBUG] Missing-value handling complete.


In [6]:
# # =============================
# # 6. OUTLIER DETECTION (OPTIONAL)
# # =============================

# # IQR method
# def remove_iqr_outliers(df):
#     df_clean = df.copy()
#     for col in df_clean.select_dtypes(include=[np.number]):
#         Q1 = df_clean[col].quantile(0.25)
#         Q3 = df_clean[col].quantile(0.75)
#         IQR = Q3 - Q1
#         low = Q1 - 1.5*IQR
#         high = Q3 + 1.5*IQR
#         df_clean = df_clean[(df_clean[col] >= low) & (df_clean[col] <= high)]
#     return df_clean

# # remove outliers only in training (never in test)
# train_no_outliers = remove_iqr_outliers(pd.concat([X, y], axis=1))
# print("Before:", len(train), "After:", len(train_no_outliers))

# y = train_no_outliers[TARGET]
# X = train_no_outliers.drop(columns=[TARGET])


In [7]:
# =============================
# 6. SAFE ENCODING (MEMORY-SAFE)
# =============================
# Determine cardinality on training set
cardinality = {c: X[c].nunique() for c in categorical_cols}
cardinality_series = pd.Series(cardinality).sort_values(ascending=False)
print("[DEBUG] Cardinality (top):")
print(cardinality_series)

# Rules: high_cardinality > 30 -> ordinal encode; low_cardinality <= 30 -> one-hot
HIGH_CARD_LIMIT = 30
high_cardinality = [c for c, n in cardinality.items() if n > HIGH_CARD_LIMIT]
low_cardinality  = [c for c, n in cardinality.items() if n <= HIGH_CARD_LIMIT]

print("\nHigh-cardinality cols:", high_cardinality)
print("Low-cardinality cols:", low_cardinality)

# Ordinal encode high-card cols (preserve index)
if high_cardinality:
    oe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    X[high_cardinality] = oe.fit_transform(X[high_cardinality])
    test_X[high_cardinality] = oe.transform(test_X[high_cardinality])

# One-hot encode low-card cols
if low_cardinality:
    ohe = OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore")
    # Fit on train and get column names
    ohe_train = ohe.fit_transform(X[low_cardinality])
    ohe_test  = ohe.transform(test_X[low_cardinality])

    ohe_cols = ohe.get_feature_names_out(low_cardinality)

    # Build DataFrames while preserving the original row index
    ohe_train_df = pd.DataFrame(ohe_train, columns=ohe_cols, index=X.index)
    ohe_test_df  = pd.DataFrame(ohe_test,  columns=ohe_cols, index=test_X.index)

    # Drop original low_cardinality columns and concat the new ones
    X = X.drop(columns=low_cardinality)
    test_X = test_X.drop(columns=low_cardinality)

    X = pd.concat([X, ohe_train_df], axis=1)
    test_X = pd.concat([test_X, ohe_test_df], axis=1)

print("\n[DEBUG] Encoding complete. Shapes:")
print("X:", X.shape, "test_X:", test_X.shape)


[DEBUG] Cardinality (top):
FundUseCase           5
QualificationLevel    4
WorkCategory          4
RelationshipStatus    3
OwnsProperty          2
FamilyObligation      2
JointApplicant        2
dtype: int64

High-cardinality cols: []
Low-cardinality cols: ['QualificationLevel', 'WorkCategory', 'RelationshipStatus', 'OwnsProperty', 'FamilyObligation', 'FundUseCase', 'JointApplicant']

[DEBUG] Encoding complete. Shapes:
X: (204277, 25) test_X: (51070, 25)


In [8]:
# =============================
# 7. SAFE ALIGNMENT OF COLUMNS (no row alignment)
# =============================
# Add any missing columns in test_X with zeros; remove any extra columns in test_X not in X
train_cols = X.columns.tolist()
test_cols  = test_X.columns.tolist()

# Add missing cols to test_X
for c in train_cols:
    if c not in test_cols:
        test_X[c] = 0

# Remove extra cols from test_X (if any)
extra_cols = [c for c in test_X.columns if c not in train_cols]
if extra_cols:
    test_X = test_X.drop(columns=extra_cols)

# Reorder test_X columns to match X
test_X = test_X[train_cols]

print("[DEBUG] After safe column sync:")
print("X.columns == test_X.columns ?", (X.columns.tolist() == test_X.columns.tolist()))


[DEBUG] After safe column sync:
X.columns == test_X.columns ? True


In [9]:
# =============================
# 8. STANDARDIZATION
# =============================
scaler = StandardScaler()

# Note: numeric_cols list refers to original numeric columns; after encoding, those names still exist
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
test_X[numeric_cols] = scaler.transform(test_X[numeric_cols])

print("[DEBUG] Standardization complete.")


[DEBUG] Standardization complete.


In [10]:
# =============================
# 9. SAVE CLEAN PROCESSED DATASET FILES
# =============================

# Combine features + target into a single training file (VERY convenient for model notebooks)
train_processed = pd.concat([X, y], axis=1)

# Add ProfileID back into both train and test for reference (optional but useful)
train_processed.insert(0, "ProfileID", train_profile)
test_processed = test_X.copy()
test_processed.insert(0, "ProfileID", test_profile)

# Save processed datasets
train_processed.to_csv("/kaggle/working/train_processed.csv", index=False)
test_processed.to_csv("/kaggle/working/test_processed.csv", index=False)

print("Saved:")
print(" - /kaggle/working/train_processed.csv")
print(" - /kaggle/working/test_processed.csv")


Saved:
 - /kaggle/working/train_processed.csv
 - /kaggle/working/test_processed.csv


In [11]:
# =============================
# 10. SANITY CHECK (OPTIONAL)
# =============================
train2 = pd.read_csv("/kaggle/working/train_processed.csv")
test2  = pd.read_csv("/kaggle/working/test_processed.csv")

print("Reloaded shapes:")
print("train_processed:", train2.shape)
print("test_processed :", test2.shape)

print("\n[DEBUG] y distribution in reloaded train_processed:")
print(train2["RiskFlag"].value_counts())


Reloaded shapes:
train_processed: (204277, 27)
test_processed : (51070, 26)

[DEBUG] y distribution in reloaded train_processed:
RiskFlag
0    180524
1     23753
Name: count, dtype: int64


In [12]:
# # =============================
# # 11. STANDARDIZATION
# # =============================
# scaler = StandardScaler()

# X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
# test_copy[numeric_cols] = scaler.transform(test_copy[numeric_cols])


In [13]:
# # =============================
# # 12. LOW VARIANCE FILTER
# # =============================

# # Convert all column names to strings
# X.columns = X.columns.astype(str)
# test_copy.columns = test_copy.columns.astype(str)

# selector = VarianceThreshold(threshold=0.0)
# selector.fit(X)

# selected_cols = X.columns[selector.get_support()]

# X = X[selected_cols]
# test_copy = test_copy[selected_cols]

# print("Remaining features:", X.shape[1])

# # 🔍 DEBUG 5 — Final check before saving
# print("\n[DEBUG] FINAL RiskFlag counts (should match raw):")
# print(y.value_counts())


In [14]:
# # =============================
# # 13. SAVE FINAL PROCESSED DATA
# # =============================
# X.to_csv("/kaggle/working/X_processed.csv", index=False)
# test_copy.to_csv("/kaggle/working/test_processed.csv", index=False)
# y.to_csv("/kaggle/working/y.csv", index=False)

# print("Processing completed.")
